## Libraries

In [1]:
import matplotlib
matplotlib.use('TkAgg')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from new_datasets_py import create_subsets

## Load file

In [2]:
# Load the data
data = pd.read_csv('crypto-markets.csv')
btc_data = data[data['symbol'] == 'BTC'].copy()
doge_data = data[data['symbol'] == 'DOGE'].copy()
ltc_data = data[data['symbol'] == 'LTC'].copy()
btc_data.loc[:, 'date'] = pd.to_datetime(btc_data['date'])
doge_data.loc[:, 'date'] = pd.to_datetime(doge_data['date'])
ltc_data.loc[:, 'date'] = pd.to_datetime(ltc_data['date'])
btc_data.set_index('date', inplace=True)
doge_data.set_index('date', inplace=True)
ltc_data.set_index('date', inplace=True)
#missing_values = btc_data.isnull().sum() + doge_data.isnull().sum() + ltc_data.isnull().sum()
#print("Missing values:\n", missing_values)
cr_subsets = create_subsets()

/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(seq

## Creating subsets

In [3]:
btc_subsets = cr_subsets.create_random_subsets(btc_data)
doge_subsets = cr_subsets.create_random_subsets(doge_data)
ltc_subsets = cr_subsets.create_random_subsets(ltc_data)

In [4]:
btc_table = cr_subsets.create_dataset_table(btc_subsets)
doge_table = cr_subsets.create_dataset_table(doge_subsets)
ltc_table = cr_subsets.create_dataset_table(ltc_subsets)

In [5]:
btc_table_with_label = cr_subsets.add_label_column(btc_table)
ltc_table_with_label = cr_subsets.add_label_column(ltc_table)
doge_table_with_label = cr_subsets.add_label_column(doge_table)

print(doge_table_with_label)

    dataset_id                                              dates  \
0            1  [2018-03-04, 2018-03-05, 2018-03-06, 2018-03-0...   
1            2  [2014-09-20, 2014-09-21, 2014-09-22, 2014-09-2...   
2            3  [2015-05-15, 2015-05-16, 2015-05-17, 2015-05-1...   
3            4  [2014-04-13, 2014-04-14, 2014-04-15, 2014-04-1...   
4            5  [2018-05-08, 2018-05-09, 2018-05-10, 2018-05-1...   
5            6  [2017-07-05, 2017-07-06, 2017-07-07, 2017-07-0...   
6            7  [2015-06-21, 2015-06-22, 2015-06-23, 2015-06-2...   
7            8  [2017-05-14, 2017-05-15, 2017-05-16, 2017-05-1...   
8            9  [2015-06-02, 2015-06-03, 2015-06-04, 2015-06-0...   
9           10  [2015-02-24, 2015-02-25, 2015-02-26, 2015-02-2...   
10          11  [2015-01-19, 2015-01-20, 2015-01-21, 2015-01-2...   
11          12  [2014-07-15, 2014-07-16, 2014-07-17, 2014-07-1...   
12          13  [2016-12-17, 2016-12-18, 2016-12-19, 2016-12-2...   
13          14  [2017-06-16, 2017-

## Arima init

In [30]:
from statsmodels.tsa.arima.model import ARIMA

rise_result = []
combined_table = pd.concat([doge_table_with_label, ltc_table_with_label, btc_table_with_label])

TP, TN, FP, FN = 0, 0, 0, 0

for index, row in combined_table.iterrows():
    training_data = row['close_values'][:7]
    model = ARIMA(training_data, order=(2,2,1))
    model_fit = model.fit()

    forecast_test = model_fit.forecast(3)

    predicted_rise = 1 if forecast_test[2] > row['close_values'][9] else 0
    actual_rise = row['rise']

    if predicted_rise == 1 and actual_rise == 1:
        TP += 1
    elif predicted_rise == 0 and actual_rise == 0:
        TN += 1
    elif predicted_rise == 1 and actual_rise == 0:
        FP += 1
    elif predicted_rise == 0 and actual_rise == 1:
        FN += 1

print("True Positives (TP):", TP)
print("True Negatives (TN):", TN)
print("False Positives (FP):", FP)
print("False Negatives (FN):", FN)

# btc_data_daily['close'].plot(label='Original Data')
# print(forecast_test)

# data[data['symbol'] == 'BTC'].copy()
# model = ARIMA(training_data, order=(4,2,1))
# model_fit = model.fit()
# print(model_fit.summary())

/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model

True Positives (TP): 17
True Negatives (TN): 20
False Positives (FP): 45
False Negatives (FN): 38


/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [31]:
sensitivity = TP / (TP + FN)
specificity = TN / (FP + TN)
precision = TP / (TP + FP)
accuracy = (TP + TN) / (TP + TN + FP + FN)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision", precision)
print("Accuracy", accuracy)

Sensitivity: 0.3090909090909091
Specificity: 0.3076923076923077
Precision 0.27419354838709675
Accuracy 0.30833333333333335


In [33]:
X = []
Y = []
X = combined_table['close_values'].apply(lambda x: x[:7]).tolist()
Y = combined_table['rise'].astype(int).tolist()
X_array = np.array(X)
Y_array = np.array(Y)

print(X)
print(X_array.shape, Y_array)


[[0.005502, 0.005218, 0.00461, 0.004024, 0.004025, 0.004146, 0.003973], [0.000292, 0.000288, 0.000298, 0.000313, 0.000348, 0.000375, 0.000451], [0.00012, 0.000117, 0.000115, 0.000116, 0.00012, 0.000134, 0.000134], [0.000389, 0.000461, 0.000587, 0.000708, 0.000643, 0.000617, 0.000646], [0.005303, 0.005093, 0.00487, 0.004362, 0.004473, 0.004329, 0.00446], [0.002714, 0.002636, 0.002397, 0.00244, 0.00233, 0.001952, 0.001779], [0.000209, 0.000219, 0.000187, 0.000178, 0.000186, 0.000201, 0.00021], [0.001274, 0.001128, 0.001059, 0.001318, 0.001508, 0.001518, 0.0019], [0.00016, 0.000151, 0.000152, 0.000153, 0.000158, 0.000152, 0.000163], [0.00014, 0.000142, 0.000143, 0.000146, 0.000145, 0.000146, 0.000149], [0.00014, 0.000141, 0.000153, 0.000155, 0.000156, 0.000166, 0.000166], [0.000254, 0.000234, 0.000261, 0.000247, 0.000245, 0.00024, 0.000237], [0.000219, 0.000218, 0.000218, 0.000219, 0.000227, 0.000225, 0.000236], [0.003163, 0.003322, 0.00323, 0.003141, 0.003218, 0.003125, 0.003104], [0.003

In [35]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import balanced_accuracy_score
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)

for train, test in rskf.split(X_array, Y_array):
    model = MLPClassifier()
    model.fit(X_array[train], Y_array[train])
    y_pred = model.predict(X_array[test])
    acc = balanced_accuracy_score(y_pred, Y_array[test])

    print(acc)
    


0.4246031746031746
0.2627118644067797


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


# Define LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_array.shape[1], 1)))
model.add(Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.2)

# # Evaluate model
# loss, accuracy = model.evaluate(X_test, Y_test)
# print(f'Loss: {loss}, Accuracy: {accuracy}')

# # Make predictions
# predictions = model.predict(X_test)

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Argument `validation_split` is only supported for tensors or NumPy arrays.Found incompatible type in the input: [<class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class '